Dataset: https://www.kaggle.com/c/fake-news/data#

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv('/content/train.csv')

In [5]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [6]:
df.isnull().sum()

,0
id,0
title,558
author,1957
text,39
label,0


In [7]:
# drop Nan values

df = df.dropna()

In [8]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [9]:
# get the independent features

x = df.drop('label', axis=1)

In [10]:
# get the dependent features

y = df['label']

In [11]:
x.shape

(18285, 4)

In [12]:
import tensorflow as tf

In [13]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [14]:
# vocabulary size

voc_size = 5000

# Onehot Representation

In [15]:
messages = x.copy()

In [16]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [17]:
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [18]:
messages.reset_index(inplace=True)

In [19]:
messages

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...,...
18280,20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
18281,20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
18282,20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
18283,20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [20]:
import nltk
import re
from nltk.corpus import stopwords

In [21]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [22]:
# dataset preprocessing

from nltk.stem.porter import PorterStemmer

ps = PorterStemmer()
corpus = []

for i in range(0, len(messages)):
  review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
  review = review.lower()
  review = review.split()

  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

In [23]:
 corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [24]:
onehot_repr = [one_hot(words, voc_size) for words in corpus]
onehot_repr

[[4084, 3769, 1476, 4565, 2964, 2465, 2759, 642, 2248, 2912],
 [1337, 4128, 2938, 355, 3041, 899, 2150],
 [501, 1186, 4178, 2685],
 [2619, 3689, 4684, 837, 3084, 936],
 [4802, 3041, 1290, 951, 2620, 381, 3041, 2912, 728, 3480],
 [4249,
  944,
  1584,
  1711,
  4185,
  752,
  3879,
  3993,
  3237,
  3733,
  1906,
  4001,
  4159,
  3572,
  2150],
 [383, 3187, 1054, 4854, 2553, 3065, 4728, 1192, 4711, 1363, 2709],
 [888, 4435, 521, 1726, 2416, 2967, 752, 4195, 4711, 1363, 2709],
 [887, 1402, 2756, 712, 4488, 3912, 4038, 3278, 752, 341],
 [4703, 4793, 2319, 3098, 1616, 1926, 2499, 1374],
 [1808, 434, 2567, 4578, 1208, 3503, 868, 2792, 4400, 4545, 4071],
 [837, 110, 2964, 3912, 752, 2416],
 [2788, 3074, 2305, 4212, 3583, 888, 175, 3357, 1234],
 [46, 2845, 3296, 4225, 2619, 2865, 671, 4711, 1363, 2709],
 [4837, 272, 1807, 2216, 3493, 4711, 1363, 2709],
 [359, 566, 2605, 3865, 4071, 190, 3260, 4519, 1870, 2181],
 [4647, 1903, 4128],
 [4133, 4935, 1019, 2839, 752, 374, 2584, 2150],
 [909, 1218

# Embedding Representation

In [25]:
sent_length = 20
embedded_docs = pad_sequences(onehot_repr, padding='pre', maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ...  642 2248 2912]
 [   0    0    0 ... 3041  899 2150]
 [   0    0    0 ... 1186 4178 2685]
 ...
 [   0    0    0 ... 4711 1363 2709]
 [   0    0    0 ...  670 4551 2597]
 [   0    0    0 ... 3698 2169  997]]


In [26]:
embedded_docs[1]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0, 1337, 4128, 2938,  355, 3041,  899, 2150], dtype=int32)

In [27]:
# create model

embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [28]:
import numpy as np
x_final = np.array(embedded_docs)
y_final = np.array(y)

In [29]:
x_final.shape, y_final.shape

((18285, 20), (18285,))

In [30]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_final, y_final, test_size=0.33, random_state=42)

# Model Training

In [32]:
# Model Training

model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=64)

Epoch 1/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 12s 49ms/step - accuracy: 0.7809 - loss: 0.4309 - val_accuracy: 0.9105 - val_loss: 0.1982
Epoch 2/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 6s 32ms/step - accuracy: 0.9405 - loss: 0.1488 - val_accuracy: 0.9203 - val_loss: 0.1929
Epoch 3/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 8s 43ms/step - accuracy: 0.9624 - loss: 0.1000 - val_accuracy: 0.9138 - val_loss: 0.2139
Epoch 4/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 6s 31ms/step - accuracy: 0.9737 - loss: 0.0731 - val_accuracy: 0.9180 - val_loss: 0.2357
Epoch 5/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 12s 41ms/step - accuracy: 0.9827 - loss: 0.0552 - val_accuracy: 0.9074 - val_loss: 0.2894
Epoch 6/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 11s 45ms/step - accuracy: 0.9869 - loss: 0.0413 - val_accuracy: 0.9133 - val_loss: 0.2984
Epoch 7/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 8s 32ms/step - accuracy: 0.9909 - loss: 0.0297 - val_accuracy: 0.9165 - val_loss: 0.3241
Epoch 8/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 10s 32ms/step - accuracy: 0.9938 - loss: 0.0226 - val_

# Performance Metrics and Accuracy

In [33]:
y_pred = model.predict(x_test)

189/189 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step


In [35]:
y_pred=np.where(y_pred > 0.5, 1, 0)  #AUC ROC Curve

In [36]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred)

array([[3129,  290],
       [ 247, 2369]])

In [37]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_pred)

0.9110190555095278

In [38]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.92      0.92      3419
           1       0.89      0.91      0.90      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035

